In [29]:
# The aim for this project is to be able to simply model the characteristics of a semiconductor device given a number
# of properties of the device

In [30]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy import constants as spcon
from plotly.subplots import make_subplots

In [93]:
#Define the scientific constants necessary for the modelling
q = spcon.elementary_charge #units of C
perm0 = spcon.epsilon_0 #units of F/m
kB = spcon.k / (1.602177 * (10 **-19)) #units of eV/K

#Define the material properties relevant to this specific system
permS = 11.68
semiWF = 4.47 #units of eV
metalWF = 4.88 #units of eC
dopeConc = (1 * 10 ** 20) #in units of cm^-3
hMob = 450 #units of cm^2/V.s

#Define the device parameters, such as gate length, gate width and channel depth
lGate = 20 * 10 ** (-4) #in units of cm
wGate = 200 * 10 ** (-4) #in units of cm
dChan = 100 * 10 ** (-8) #in units of cm
glMod = 1/1000

#Define the temperatures in K that the initial measurement and the simulation are run at
tMeas = 298
tSim = 350

In [32]:
#Calculate the built in voltage for the device
vBI = (metalWF - semiWF)

The built-in voltage of the MESFET is calculated using:
$$V_{bi} = \sigma_{m} - \sigma_{s}$$

---

The pinch-off voltage is calculated using:
$$V_P = q\cdot\frac{N_a\cdot d^2}{2\cdot\epsilon_S\cdot\epsilon_0}$$

In [94]:
#Define the input array for the current calculation
inputs = {'q':q,
          'permS':permS,
          'perm0':perm0,
          'hMob':hMob,
          'wGate':wGate,
          'dopeConc':dopeConc,
          'dChan':dChan,
          'lGate':lGate,
          'vBI':vBI,
          'tMeas':tMeas,
          'tSim':tSim,
          'glMod':glMod
          }

In [40]:
#Calculate the depth of the depletion region in the MESFET (ie the region in which current is restricted)
#Consider the built-in voltage, the gate voltage and the drain voltage
def TotDepletionDepth(par, vG, vD):
    depth = np.sqrt(((2*par['permS']*par['perm0'])/(par['q']*par['dopeConc']))*(par['vBI'] + vG + vD))
    return depth

The total depletion depth is given by

$$depth = \sqrt{\frac{2\epsilon_S\epsilon_0}{qN_a}\cdot (V_{bi} + V_G + V_D)}$$

In [69]:
def idealCoeff(par):
    beta = (2 * par['q'] * par['hMob'] * par['wGate'] * par['dopeConc'] * par['dChan']) / par['lGate']
    vT = -(par['vBI']-((par['q']*par['dopeConc']*(par['dChan'])**2)/(2*par['permS']*par['perm0'])))
    return beta, vT

In [205]:
def tempCoeff(par):
    beta = ((2 * par['q'] * par['hMob'] * par['wGate'] * par['dopeConc'] * par['dChan']) / par['lGate']) *((par['tSim']/par['tMeas'])**-1.5)
    vT = -(par['vBI']-((par['q']*par['dopeConc']*(par['dChan'])**2)/(2*par['permS']*par['perm0']))) +0.0001*(par['tSim'] - par['tMeas'])
    return beta, vT

In [184]:
def forwardOp(beta, glMod, threshV, drainV, gateV):
    if -gateV <= -threshV:
        return 0
    elif -drainV < (-gateV+threshV):
        return beta*drainV*(2*(-gateV+threshV)+drainV)*(1-(glMod*drainV))
    elif (-gateV+threshV) <= -drainV:
        return -beta*((-gateV+threshV)**2)*(1-(glMod*drainV))
    else:
        return np.nan

In [185]:
def reverseOp(beta, glMod, threshV, drainV, gateV):
    if -gateV <= -threshV:
        return 0
    elif drainV < (-gateV+threshV):
        return beta*drainV*(2*(-gateV+threshV)-drainV)*(1+(glMod*drainV))
    elif (-gateV+threshV) <= drainV:
        return beta*((-gateV+threshV)**2)*(1+(glMod*drainV))
    else:
        return np.nan

In [194]:
def drainCurr(temp, par, drainV, gateV):
    if temp == True:
        beta, threshV = tempCoeff(par)
    else:
        beta, threshV = idealCoeff(par)
    
    if -drainV < 0:
        return reverseOp(beta, par['glMod'], threshV, drainV, gateV)
    else:
        return forwardOp(beta, par['glMod'], threshV, drainV, gateV)

Schiman and Hodges equations according to the MATHWORKS website. https://uk.mathworks.com/help/physmod/sps/ref/pchanneljfet.html

---

For forward operation ($-V_D<0$):
\
If $-V_G\leq-V_{TH}$:
$$I_D=0$$
If $-V_D>0$ and $-V_D<(-V_G+V_{TH})$:
$$I_D=\frac{2q \mu W N d}{L}V_D(2(-V_G+V_{TH})+V_D)(1-\lambda V_D)$$
If $(-V_G+V_{TH})>0$ and $(-V_G+V_{TH}) \leq -V_D$:
$$I_D=-\frac{2q \mu W N d}{L}(-V_G+V_{TH})^2(1-\lambda V_D)$$

---

For reverse operation ($-V_D>0$):
\
If $-V_G\leq-V_{TH}$:
$$I_D=0$$
If $V_D>0$ and $V_D<(-V_G+V_{TH})$:
$$I_D=\frac{2q \mu W N d}{L}V_D(2(-V_G+V_{TH})-V_D)(1+\lambda V_D)$$
If $(-V_G+V_{TH})>0$ and $(-V_G+V_{TH}) \leq V_D$:
$$I_D=\frac{2q \mu W N d}{L}(-V_G+V_{TH})^2(1+\lambda V_D)$$

In [183]:
numStep = 50

In [206]:
gateV = np.linspace(-4, 1, numStep)
drainV = np.linspace(-4, 4, numStep)
column_names = np.array(['Drain Voltage [V]', 'Gate Voltage [V]', 'Drain Current [A]', 'Depletion Depth'])
df = pd.DataFrame(columns = column_names)

for i in range(numStep):
    for j in range(numStep):
        drainI = drainCurr(False, inputs, drainV[i], gateV[j])
        newData = np.array([[drainV[i], gateV[j], drainI, TotDepletionDepth(inputs, gateV[i], drainV[j])]])
        df2 = pd.DataFrame(newData, columns = column_names)
        df = df.append(df2, ignore_index=True)

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning:

invalid value encountered in sqrt



In [207]:
df2 = df[['Drain Voltage [V]',
          'Gate Voltage [V]',
          'Drain Current [A]'
         ]].astype(float)

plotting = df2.pivot_table(index = 'Drain Voltage [V]',
                           columns = 'Gate Voltage [V]',
                           values = 'Drain Current [A]'
                          )

fig = go.Figure(go.Heatmap(z=plotting.values,
                           x=plotting.columns,
                           y=plotting.index,
                           colorscale='rdbu',
                           reversescale=True,
                           colorbar={"title": "Drain Current [A]"},
                          )
               )
fig.update_layout(
    title="Drain Current [A] for various Gate and Drain voltages",
    xaxis=dict(title="Gate Voltage [V]",showline=True, linewidth=2, linecolor='black', mirror=True),
    yaxis=dict(title="Drain Voltage [V]", showline=True, linewidth=2, linecolor='black', mirror=True),
    font=dict(
        size=14,
        color="#7f7f7f"
    )
)

fig.write_html('Currents_Heatmap.html')
fig.show()

In [87]:
df3 = df[['Drain Voltage [V]',
          'Gate Voltage [V]',
          'Depletion Depth'
         ]].astype(float)
plotting = df3.pivot_table(index = 'Drain Voltage [V]',
                           columns = 'Gate Voltage [V]',
                           values = 'Depletion Depth'
                          )

fig = go.Figure(go.Heatmap(z=plotting.values,
                           x=plotting.columns,
                           y=plotting.index,
                           colorscale= 'rdbu',
                           reversescale=True,
                           colorbar={"title": 'Depletion Depth'}
                          )
               )

fig.update_layout(
    title="Depletion Depth for various Gate and Drain voltages",
    xaxis=dict(title="Gate Voltage [V]",showline=True, linewidth=2, linecolor='black', mirror=True),
    yaxis=dict(title="Drain Voltage [V]", showline=True, linewidth=2, linecolor='black', mirror=True),
    font=dict(
        size=14,
        color="#7f7f7f"
    ))

fig.write_html('Depletion_Heatmap.html')
fig.show()

In [210]:
fig = go.Figure()

prim_y_vals=np.empty_like(drainV)
fig = make_subplots(specs=[[{"secondary_y": True}]])

for step in gateV:
    for i in range(numStep):
        prim_y_vals[i] = drainCurr(False, inputs, drainV[i], step)
        
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(width=2),
            name="Gate voltage = " + "{:.2f}".format(step),
            x=drainV,
            y=prim_y_vals,
        ),
    )

fig.data[0].visible = True

# Create and add slider
steps = []
for i in range(len(gateV)):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
    )
    step["args"][1][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Drain Voltage: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    title='Drain current against drain voltage for given gate voltage',
    legend=dict(x=-0.1, y=-0.7),
    showlegend=True
)

fig.update_layout(
    title="Drain Current vs Drain Voltage",
    xaxis=dict(title='Drain Voltage [V]', showline=True, linewidth=2, linecolor='black', mirror=True),
    yaxis=dict(title='Drain Current [A]', showline=True, linewidth=2, linecolor='black', mirror=True),
    font=dict(
        size=14,
        color="#7f7f7f"
    )
)

fig.show()
fig.write_html("CurrentAgainstDrainV.html")

In [208]:
fig = go.Figure()

prim_y_vals=np.empty_like(drainV)

for step in gateV[::int(numStep/10)]:
    for i in range(numStep):
        prim_y_vals[i] = drainCurr(True, inputs, drainV[i], step)
        
    fig.add_trace(
        go.Scatter(
            visible=True,
            line=dict(width=2),
            name="Gate Voltage = " + "{:.2f}".format(step),
            x=drainV,
            y=prim_y_vals,
        ),
    )

fig.update_layout(
    title="Drain Current vs Drain Voltage",
    xaxis=dict(title='Drain Voltage [V]', showline=True, linewidth=2, linecolor='black', mirror=True),
    yaxis=dict(title='Drain Current [A]', showline=True, linewidth=2, linecolor='black', mirror=True),
    font=dict(
        size=14,
        color="#7f7f7f"
    )
)

fig.show()
fig.write_html("CurrentAgainstDrainV.html")